In [1]:
# import python built-in libraries
import os.path

# import external libraries
import netCDF4 
import pandas as pd
import numpy as np
import yaml
from datetime import datetime, timedelta
from cftime import num2date, date2num

# import user libraries
from utils import description_module
from utils import conversion_module

# load configuration file
with open('config.yml') as f:
    cfg = yaml.load(f, Loader = yaml.FullLoader)

In [2]:


dataPath = "./data/"
# dataPath = "./raw_data/Maebong_and_Moonam/"
 
files = os.listdir(dataPath)

datasetDict = {}
for idx,nc_type_file in enumerate(files):
    if nc_type_file.split(".")[-1] == 'nc':
        datasetDict[nc_type_file] = netCDF4.Dataset(dataPath+nc_type_file,mode="r")

In [3]:
ncfile_name = '20210731_test_example_2.nc'
output_dataset = datasetDict[ncfile_name]

# 하단 데이터를 갖고 있습니다.
output_dataset.variables.keys()

#mdww : mean direction of wind waves
#mpww : mean period of wind waves
#mwd : mean wave direction
#mwp : mean wave period
#wmb : Model bathymetry
#dwi : Ocean surface stress equivalent 10m neutral wind direction
#swh : Significant height of combined wind waves and swell
#shww : significant height of wind waves

dict_keys(['longitude', 'latitude', 'time', 'u100', 'v100'])

In [4]:
# 차원이 다르게 받아들여지는 경우는 해결해야 함...;

In [5]:
a, b = conversion_module.netCDF4_to_CSV(output_dataset)

In [18]:
list(a.items())[1]

('u100',
 array([[1.19920014],
        [1.57312448],
        [1.1999525 ],
        ...,
        [1.53099216],
        [1.78378609],
        [2.29238339]]))

In [6]:
a['time'].shape

(26304, 1)

In [24]:
len(list(a.items())[0][1])

26304

In [20]:
len(list(a.items())[0,1])

TypeError: list indices must be integers or slices, not tuple

In [19]:
len(a)

3

In [9]:
a['v100'][0]

array([-3.38774517])

In [10]:

output_df = pd.DataFrame(np.concatenate((a['time'], a['v100']),axis=1))
output_df

,0,1
0,2018-01-01 00:00:00,-3.387745165685549
1,2018-01-01 01:00:00,-3.3271057486084805
2,2018-01-01 02:00:00,-4.063085522995914
3,2018-01-01 03:00:00,-4.451842333846161
4,2018-01-01 04:00:00,-4.10877275504028
...,...,...
26299,2020-12-31 19:00:00,-5.925463236513548
26300,2020-12-31 20:00:00,-4.996766410593515
26301,2020-12-31 21:00:00,-3.8313266549890357
26302,2020-12-31 22:00:00,-3.0945162036553415


In [ ]:
a['v100'].reshape(26304,-1).tolist()

In [ ]:
a.keys()

In [ ]:
a

In [ ]:
# 하단에 저장되어 있습니다.
datasetDict.keys()

In [ ]:
masked_array_dtime = netCDF4.num2date(output_dataset['time'],output_dataset['time'].units)

In [ ]:
len(masked_array_dtime)

In [ ]:
len(output_dataset['time'])

In [ ]:
output_dataset['time'].get_dims()

In [ ]:
a = list(output_dataset['time'].get_dims()[0])

In [ ]:
a

In [ ]:
output_dataset['time']

In [ ]:
output_dataset

In [ ]:
output_dataset['time']

In [ ]:
output_dataset['longitude']

output_dataset['time'].get_dims()[0].size

In [ ]:
# t2m = output_dataset.variables
time_dim, lat_dim, lon_dim = output_dataset['time'].get_dims()

In [ ]:
output_dataset['u10'].get_dims()

In [ ]:
# output_dataset.variables

In [ ]:
output_dataset.variables['time']

In [ ]:
output_dataset['time']

In [ ]:
t2m

In [ ]:
output_dataset.variables['expver']

In [ ]:
timedim = output_dataset['time'].get_dims()[0].size

time_var = output_dataset.variables[time_dim.name]
times = num2date(time_var[:], time_var.units)


In [ ]:
time_var = output_dataset.variables['time']
times = num2date(time_var[:], time_var.units)


In [ ]:
str(times[0])

In [ ]:
time_var.units

In [ ]:
meta_data

In [ ]:
output_dataset = datasetDict['20210731_wando.nc']
output_dataset.variables.keys()

In [ ]:
target_dataset = '20201114_testdata.nc'
output_dataset = datasetDict[target_dataset]

output_dataset.variables.keys()

In [ ]:
len(output_list)

In [ ]:
output_dataset['time']

In [ ]:
times = [str(i) for i in netCDF4.num2date(output_dataset['time'],output_dataset['time'].units)]

times = np.array(times)

len(times)

In [ ]:
times

In [ ]:
times =  times.reshape(len(times),1)
aims = []
aims.append(times)

for i in output_list[1:]:
    aims.append(i[:,0,0].reshape(len(i[:,0,0]),1))

In [ ]:
output_df = pd.DataFrame(np.concatenate(aims,axis=1), columns =column_name)

# output_df = pd.DataFrame(np.concatenate(output_list,axis=1).reshape(17520,5),columns=column_name)
output_df.head()

In [ ]:
from metpy.calc import wind_speed, wind_direction
from metpy.units import units

def recordFunction(record):
    u = float(record['u10']) * units.meter / units.second
    v = float(record['v10']) * units.meter / units.second
    return [wind_direction(u,v), wind_speed(u,v)]

In [ ]:
listofTwo = output_df.apply(recordFunction, axis=1)

# type(listofTwo.tolist()[0][0])

# listofTwo.tolist()[0][0].to_compact

a = pd.DataFrame(listofTwo.tolist(),columns =['direction','wind_speed'])

column_name.append('wind_direction')
column_name.append('wind_speed')
column_name

In [ ]:
output_df1 = pd.DataFrame(np.concatenate([output_df,a],axis=1),columns = column_name)

file_name = str(meta_data[0][0])+"(lat) "+str(meta_data[1][0])+"(lon)1 located data"+'.csv'
output_df1.to_csv(file_name,index=True, header=True)

In [ ]:
file_name

In [ ]:
 a = pd.DataFrame(listofTwo.tolist(),columns =['direction','wind_speed'])

In [ ]:
file_name = str(meta_data[0][0])+"(lat) "+str(meta_data[1][0])+"(lon)1 located data"+'.csv'
# output_df.to_csv(file_name,index=True, header=True)

In [ ]:
output_df1.to_csv(file_name,index=True, header=True)

In [ ]:
pd.concat([output_df,a],axis=1).to_csv(file_name,index=True, header=True)

In [ ]:
output_df.shape

In [ ]:
output_dataset['time'].units

In [ ]:
meta_data[0][0]

In [ ]:
meta_data[1][0]